In [ ]:
import requests
import pandas as pd
import time
import requests_cache
import re
from bs4 import BeautifulSoup
session = requests_cache.CachedSession('project_cache')

In [ ]:
zipcode = "44413"
#date = "2023-03-10"
dates = "2023-03-09", "2023-03-10", "2023-03-11"

In [ ]:
url = "https://www.airnowapi.org/aq/observation/zipCode/historical/" + "?format=application/xml" +"&zipCode=" + \
"44413" + "&date=" + date + "T00-0000"+"&distance=25" + "&api_key=6DF75E4D-6FD6-4147-80D4-823042F30E27"

In [ ]:
response = requests.get(url)

In [ ]:
# Now create a loop that takes in a range of values and outputs the result
def get_data(dates):
    url = "https://www.airnowapi.org/aq/observation/zipCode/historical/" + "?format=application/xml" +"&zipCode=" + \
    "44413" + "&date=" + dates + "T00-0000"+"&distance=25" + "&api_key=6DF75E4D-6FD6-4147-80D4-823042F30E27"
    response = requests.get(url)
    
    return response.text

In [ ]:
get_data_output = ''
for i in dates: 
    get_data_output += get_data(i) #extend array with each iteration  

In [ ]:
len(get_data_output)

2718

In [ ]:
xml = BeautifulSoup(get_data_output, 'xml')

In [ ]:
print(xml.prettify())

<?xml version="1.0" encoding="utf-8"?>
<ObsByZipList>
 <ObsByZip>
  <DateObserved>
   03/09/2023 12:00:00 AM
  </DateObserved>
  <HourObserved>
   0
  </HourObserved>
  <LocalTimeZone>
   EST
  </LocalTimeZone>
  <ReportingArea>
   Youngstown
  </ReportingArea>
  <StateCode>
   OH
  </StateCode>
  <Latitude>
   41.0992
  </Latitude>
  <Longitude>
   -80.6464
  </Longitude>
  <ParameterName>
   OZONE
  </ParameterName>
  <AQI>
   38
  </AQI>
  <CategoryNumber>
   1
  </CategoryNumber>
  <CategoryName>
   Good
  </CategoryName>
 </ObsByZip>
 <ObsByZip>
  <DateObserved>
   03/09/2023 12:00:00 AM
  </DateObserved>
  <HourObserved>
   0
  </HourObserved>
  <LocalTimeZone>
   EST
  </LocalTimeZone>
  <ReportingArea>
   Youngstown
  </ReportingArea>
  <StateCode>
   OH
  </StateCode>
  <Latitude>
   41.0992
  </Latitude>
  <Longitude>
   -80.6464
  </Longitude>
  <ParameterName>
   PM2.5
  </ParameterName>
  <AQI>
   23
  </AQI>
  <CategoryNumber>
   1
  </CategoryNumber>
  <CategoryName>
   

In [ ]:
# Below is just a bunch of data processing and putting it together

In [ ]:
xml = BeautifulSoup(response.text, 'xml')

In [ ]:
AQI = xml.find_all('AQI')
AQI

[<AQI>36</AQI>, <AQI>19</AQI>]

In [ ]:
AQI = str(AQI)
AQI_values = re.findall(r'<AQI>(\d+)</AQI>', AQI)
AQI_values

['36', '19']

In [ ]:
Date = xml.find_all('DateObserved')
Date

[<DateObserved>03/10/2023 12:00:00 AM</DateObserved>,
 <DateObserved>03/10/2023 12:00:00 AM</DateObserved>]

In [ ]:
Date = str(Date)

In [ ]:
Date_values = re.findall(r'<DateObserved>(\d{2}/\d{2}/\d{4})', Date)

In [ ]:
Date

'[<DateObserved>03/10/2023 12:00:00 AM</DateObserved>, <DateObserved>03/10/2023 12:00:00 AM</DateObserved>]'

In [ ]:
Area = xml.find_all('ReportingArea')
Area

[<ReportingArea>Youngstown</ReportingArea>,
 <ReportingArea>Youngstown</ReportingArea>]

In [ ]:
Area_values = re.findall(r'<ReportingArea>(.*?)</ReportingArea>', str(Area))

In [ ]:
Area_values

['Youngstown', 'Youngstown']

In [ ]:
data = {'date': [Date_values[0]],
'AQI_Ozone': [AQI_values[0]],
'AQI_PM2.5': [AQI_values[1]],
'Location': [Area_values[0]]}

In [ ]:
data

{'date': ['03/10/2023'],
 'AQI_Ozone': ['36'],
 'AQI_PM2.5': ['19'],
 'Location': ['Youngstown']}

In [ ]:
data_df = pd.DataFrame(data,columns=['date','AQI_Ozone','AQI_PM2.5','Location'])
data_df


,date,AQI_Ozone,AQI_PM2.5,Location
0,03/10/2023,36,19,Youngstown
